<div class='heading'>
    <div style='float:left;'><h1>CPSC 4300/6300: Applied Data Science</h1></div>
    <img style="float: right; padding-right: 10px; width: 65px" src="https://bsethwalker.github.io/assets/img/clemson_paw.png">
</div>

## Course Project (Online Workers) - Checkpoint 1

**Clemson University**<br>
**Fall 2023**<br>
**Instructor(s):** Nina Hubig <br>
**Project Team:**
<ul>
    <li>David Croft <dcroft@g.clemson.edu></li>
    <li>Stephen Becker <sgbecke@g.clemson.edu></li>
    <li>Tony Hang <qhang@g.clemson.edu></li>
    <li>Zachary Trabookis <ztraboo@clemson.edu></li>
</ul>

---



In [263]:
## RUN THIS CELL TO GET THE RIGHT FORMATTING 
import requests
from IPython.core.display import HTML
styles = requests.get("https://bsethwalker.github.io/assets/css/cpsc6300.css").text
HTML(styles)

## Summary Goals

* Summary of the data set that, at a minimum, answers the following questions: What is the unit of analysis? How many observations in total are in the data set? How many unique observations are in the data set? What time period is covered?
  
* Brief summary of any data cleaning steps you have performed. For example, are there any particular observations / time periods / groups / etc. you have excluded?
  
* Description of outcome with an appropriate visualization technique.
  
* Description of key predictors with appropriate visualization techniques that compare predictors to the response. You should investigate all predictors in your data as part of your project. For the purpose of this assignment, pick the one or two predictors that you think are going to be most important in explaining the outcome. Your selection of predictors can either be guided by your domain knowledge or be the result of your EDA on all predictors.

In [264]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
# Set the max columns to none. This allows all the columns to display for the dataframes.
pd.set_option('display.max_columns', None)

from pandas.plotting import scatter_matrix

import statsmodels.api as sm
from statsmodels.api import OLS

from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import mean_squared_error

import warnings

In [265]:
from datetime import datetime

# Generic functions for cleaning the data
def convert_epoch_time_to_datetime(epoch_time: int):
    """
    Takes an epoch timestamp and converts it to datetime format
    Ref: 
    https://www.pythonforbeginners.com/basics/convert-epoch-to-datetime-in-python
    https://stackoverflow.com/questions/49710963/converting-13-digit-unixtime-in-ms-to-timestamp-in-python 
    """
    # Divide by 1,000 to remove ms time
    return datetime.fromtimestamp(int(epoch_time)/1000).isoformat()

# Testing the epoch time conversion
# convert_epoch_time_to_datetime(1588990000000) # '2020-05-08T22:06:40'
# convert_epoch_time_to_datetime(1588994215395)   # '2020-05-08T23:16:55.395000'


In [266]:
import json
from ast import literal_eval

import flatdict

from collections.abc import MutableMapping

def flatten_json_column(json_cols):
    """
    This function flattens JSON columns to individual columns
    It merges the flattened dataframe with expected dataframe to capture missing columns from JSON
    :param df: Crowd Work Data CSV raw dataframe
    :param json_cols: custom data columns in CSV's
    :param custom_df: expected dataframe
    :return: returns df pandas dataframe

    Ref: 
    https://github.com/vvgsrk/ParseCSVContainsJSONUsingPandas/tree/main
    https://avithekkc.medium.com/how-to-convert-nested-json-into-a-pandas-dataframe-9e8779914a24
    """

    # Make sure to sort the `na_positions` last because this could effect how many columns
    # that the nested column values are shown. If the nested column value is `NaN` first then
    # nothing will get populated for those nested column fields. (e.g. `c4_project.hit_requirements`)
    # Note: Comment out the fields that you don't want to show up in the final dataframe.
    struct_data_json = {
        "task_id": [None],
        "assignment_id": [None],
        "accepted_at": [None],
        "deadline": [None],
        "time_to_deadline_in_seconds": [None],
        "state": [None],
        "question.value": [None],
        "question.type": [None],
        # "question.attributes": [None],
        "question.attributes.FrameSourceAttribute": [None],
        "question.attributes.FrameHeight": [None],
        "project.hit_set_id": [None],
        "project.title": [None],
        "project.requester_id": [None],
        "project.requester_name": [None],
        "project.description": [None],
        "project.assignment_duration_in_seconds": [None],
        "project.creation_time": [None],
        "project.assignable_hits_count": [None],
        "project.latest_expiration_time": [None],
        "project.caller_meets_requirements": [None],
        "project.caller_meets_preview_requirements": [None],
        "project.last_updated_time": [None],
        "project.monetary_reward.currency_code": [None],
        "project.monetary_reward.amount_in_dollars": [None],
        # "project.hit_requirements.qualification_type_id": [None],
        # "project.hit_requirements.comparator": [None],
        # "project.hit_requirements.worker_action": [None],
        # "project.hit_requirements.qualification_values": [None],
        # "project.hit_requirements.caller_meets_requirement": [None],
        # "project.hit_requirements.qualification_type.qualification_type_id": [None],
        # "project.hit_requirements.qualification_type.name": [None],
        # "project.hit_requirements.qualification_type.visibility": [None],
        # "project.hit_requirements.qualification_type.description": [None],
        # "project.hit_requirements.qualification_type.has_test": [None],
        # "project.hit_requirements.qualification_type.is_requestable": [None],
        # "project.hit_requirements.qualification_type.keywords": [None],
        # "project.hit_requirements.caller_qualification_value.integer_value": [None],
        # "project.hit_requirements.caller_qualification_value.locale_value.country": [None],
        # "project.hit_requirements.caller_qualification_value.locale_subdivision": [None],
        "project.requester_url": [None],
        "expired_task_action_url": [None],
        "task_url": [None]
    }

    def _flatten_dict(d: MutableMapping, sep: str= '.') -> MutableMapping:
        """
        Take in 
        """
        [flat_dict] = pd.json_normalize(data=d, sep=sep, max_level=None).to_dict(orient='records')
        return flat_dict

    try:
        df_temp = pd.DataFrame(struct_data_json)

        # If c4 `nan` value is passed, do nothing except return empty dataframe.
        # If c4 has a string dicionary, then build new dataframe from it.
        if isinstance(json_cols, str):
            # Convert the input (str) to (dict) type.
            # Build a flattened dictionary before sending to Pandas to `json_normalize`
            dict_json_flattened = _flatten_dict(json.loads(json_cols))

            # Explicitly remove this column because it's a nested list and is hard to flatten.
            # Plus this column doesn't have any values that we need for our model.
            del dict_json_flattened["project.hit_requirements"]

            df_temp = pd.DataFrame([dict_json_flattened])
            
    except json.JSONDecodeError as e:
        print(f"flatten_json_columns: Invalid JSON argument passed - json.JSONDecodeError: {e}")

    # Return dataframe with flatten columns and 'c4.' prefix.
    return df_temp.add_prefix('c4.')


### Cleaning: Reading in the telemetry data (Amazon Mechanical Turk (AMT))
We read in and clean the data from `amazon_mechanical_turk_records.csv`.

#### Background Research Paper
Research Paper: *Quantifying the Invisible Labor in Crowd Work*

https://dl.acm.org/doi/abs/10.1145/3476060?casa_token%3Dw4mZH0IjVgsAAAAA:XBgWg_Oq0TtNVqH8SzCxl2fXU_fZ9bzQ6g22QkI0odMy5NKW2EJdYrOaqxu_2NIqJs-rA_sM1sbT

A browser plugin was used to collect the data. 
- https://github.com/GigPlatform/toloka-web-extension (has not been used for any analysis on any paper)
- https://github.com/anonym-research/invisible-labor (was used for an invisible labor analysis but any predictive analysis has been performed)

**List of available variables:** 

- **c1 (ID)**: continuous
- **c2 (current)**: url, site visited by the worker (while working)
- **c3 (event)**: categorical, 18 values (['PAGE_LOAD', 'PAGE_BLUR', 'TAB_CHANGE', 'PAGE_FOCUS', 'PAGE_CLICK', 'PAGE_SCROLL', 'PAGE_LAST', 'PAGE_CLOSE', 'INTERNALURL', 'PAGE_KEY', 'PAGE_INACTIVITY', 'TAB_CLOSED', 'EXTERNALURL', 'PAGE_REACTIVATE', 'SYSTEM_DISABLED_WORKING', 'SYSTEM_ENABLED_WORKING', 'SYSTEM_ENABLED', 'SYSTEM_DISABLED'])
  - The web browser plugin recorded multiple events, the most relevant is PAGE_LOAD, other events can provide repetitive information.
- **c4 (extra)**: json object {task_id, assignment_id, ...} – may include NaN values
  - It provides a JSON object with the specificities of the tasks, it only has values for certain events.
  - Are there any values out of this list that we should pay particular attention to?
    - Yes, I recommend parsing the JSON so you can find information about the task, including how much was paid.
- **c5 (platform)**: categorial, 5 values ['OTHER', 'MTURK', 'FIVERR', 'UPWORK', 'FREELANCER']
  - The work platform in which the worker was working on (it is usually constant)
- **c6 (skip)**: categorial, 2 values (0: no complete, 1: complete) – may include NaN values
  - That was a field that was not used
  - You mentioned skip. Does this represent if the task was completed or skipped?
    - I do not remember the purpose of that field, maybe was not used
- **c7 (subtype)**: categorial, 29 values ['OTHER', 'TASK_STARTED', 'ADDED_TASK', 'TASK_SUBMITED', 'FINISHED_TASK', 'TASKS_LIST', 'WORKER_DASHBOARD', 'UNKNOWN', 'TASK_FRAME', 'TASK_PREVIEW', 'TASK_INFO', 'TASK_RETURNED', 'PLATFORM_LOGIN', 'TASK_QUEUE', 'TASK_SKIP', 'WORKER_EARNINGS_DETAILS', 'TASK_TIMEOUT', 'WORKER_EARNINGS', 'WORKER_QUALIFICATIONS', 'TASKS_PER_REQUESTER', 'MESSAGES_SEND', 'TASKS_LIST_FILTER', 'WORKER_QUALIFICATIONS_PENDING', 'TASKS_PREVIEW', 'PLATFORM_HELP', 'TASKS_PROJECTS', 'TASKS_DETAILS', 'MESSAGES_READ', 'TASKS_APPLY']
  - It defines if a worker is listing the tasks available, if a task just started, or if a task was completed (submitted)
  - Is there a description that we can lookup to find what these event values mean?
    - FINISHED_TASK == TASK_SUBMITED both refers to task completed (submitted is when it was recently submitted and finished when the next URL was loaded)
    - TASK_RETURNED When the worker decided not to work on a task
- **c8 (time)**: continuous, datetime (milliseconds), 1970 start date Unix Time (Week, Month, Day, Hours, Minutes, Seconds)
  - This is a timestamp in milliseconds. You have to convert to a date, it contains day, month, year, hour, minute, second.
  - You can use any function that converts from timestamp to datetime.
  - What does this time represent? Task completed?
    - Time of the event, remember that this is an event log, every event happened at this time. Time series analysis is a common approach to use.
  - Does this time represent when the worker did the event recorded in c9 (type)?
    - Worker was working 
    - Communicating by sending messages in the platform 
    - Searching for tasks
    - Visiting their profile
    - That is correct. it is the time at which it was recorded
- **c9 (type)**: categorial, 10 values ['OTHER', 'WORKING', 'LOGS', 'SEARCHING', 'PROFILE', 'UNKNOWN', 'REJECTED', 'COMMUNICATION', 'LEARNING', 'PROPOSAL']
  - It identify if at that time the worker was working, communicating by sending messages in the platform, searching for tasks, visiting their profile. There are other types that mean that the workers changed the CONFIG of the web plugin or the API of Toloka retrieved new tasks.
  - Do you have a description for all these events?
    - I do not have a data dictionary but these are separated by the events from the worker in the interface: worker was working, communicating by sending messages in the platform, searching for tasks, visiting their profile. The other events do not represent an activity of the worker but a state of the plugin, does not mean any worker activity these only got extra information from the web plugin.
- **c10 (user)**: categorical, 120 values unique – Todo: Need to verify this is correct field value.
  - User ID
- **c11**: Not relevant (an activity was taken after being recommended)
- **c12**: Not used

In [267]:
def transform_time_user_to_csv(user_id):
    """
    Read in the original Amazon MT dataset and write it out for a particular user (e.g. `ae862298385abab2a0a1619f8cedef9d`)
    Convert the `c8` time column to epoch time and write out to temporary *.csv to run limited records moving forward.
    """
    # Define columns for data
    columns = ['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'time', 'c9', 'user']

    # Read the data into a dataframe

    # 4m 18.8s – Suggest writing this transformed data out to a file to read in that transformed file for further processing.
    df_amt = pd.read_csv("../data/amazon_mechanical_turk_records.csv", encoding='utf-8', header=None, names=columns)

    # Convert the epoch timestamp to datetime
    df_amt['time'] = df_amt.time.map(convert_epoch_time_to_datetime)

    df_amt = df_amt[df_amt.user == user_id]
    df_amt.to_csv(f"../data/amazon_mechanical_turk_records_{user_id}.csv",
                encoding='utf-8', header=True, columns=columns, index=False, mode="w")

In [268]:
def flatten_user_to_csv(user_id):
    """
    Read in the cleaned Amazon MT dataset and write it out for a particular user (e.g `ae862298385abab2a0a1619f8cedef9d`)
    Convert the `c4` event column by flattening most dict values into separate columns and 
    write out to temporary *.csv to run limited records moving forward.
    """
    # Read the data into a dataframe

    # Suggest writing this transformed data out to a file to read in that transformed file for further processing.
    df_amt = pd.read_csv(f"../data/amazon_mechanical_turk_records_{user_id}.csv", encoding='utf-8', header="infer")

    df_c4_flattened = pd.DataFrame()
    for i, j in df_amt.iterrows():
        if j["c4"]:
            # https://stackoverflow.com/questions/33094056/is-it-possible-to-append-series-to-rows-of-dataframe-without-making-a-list-first
            series_temp = flatten_json_column(j["c4"]).iloc[0].to_frame().T
            df_c4_flattened = pd.concat([df_c4_flattened, series_temp], ignore_index=True)
            # if i == 100:  
            #     break    

    df_amt = pd.concat([df_amt, df_c4_flattened], axis="columns")
    del df_amt["c4"]

    df_amt.to_csv(f"../data/amazon_mechanical_turk_records_{user_id}_flattened.csv",
                encoding='utf-8', header=True, index=False, mode="w")

In [272]:
user_id = "ae862298385abab2a0a1619f8cedef9d"

# Call this function to create a separate *.csv for a particular user and use that moving forward.
# transform_time_user_to_csv(user_id)

# 1m 24.3s - Flatten c4 column for user to csv.
# flatten_user_to_csv(user_id)

# Read in flattened c4 csv for the user.
df_amt = pd.read_csv(f"../data/amazon_mechanical_turk_records_{user_id}_flattened.csv", encoding='utf-8', header="infer")

# Sort by user then by time
df_amt = df_amt.sort_values(by=['user', 'time'], ascending=[True, True])

# df_amt.head(30)
df_amt[df_amt["c4.task_id"].notna()]
# df_amt.iloc[0:16,:]
# df_amt

,c1,c2,c3,c5,c6,c7,time,c9,user,c4.task_id,c4.assignment_id,c4.accepted_at,c4.deadline,c4.time_to_deadline_in_seconds,c4.state,c4.question.value,c4.question.type,c4.question.attributes.FrameSourceAttribute,c4.question.attributes.FrameHeight,c4.project.hit_set_id,c4.project.title,c4.project.requester_id,c4.project.requester_name,c4.project.description,c4.project.assignment_duration_in_seconds,c4.project.creation_time,c4.project.assignable_hits_count,c4.project.latest_expiration_time,c4.project.caller_meets_requirements,c4.project.caller_meets_preview_requirements,c4.project.last_updated_time,c4.project.monetary_reward.currency_code,c4.project.monetary_reward.amount_in_dollars,c4.project.requester_url,c4.expired_task_action_url,c4.task_url
14,15,https://worker.mturk.com/tasks/,INTERNALURL,MTURK,0.0,ADDED_TASK,2020-05-08T23:17:24.546000,LOGS,ae862298385abab2a0a1619f8cedef9d,3W9XHF7WGLV68SQR2YVUGGPI6QVTK3,3E47SOBEYRW130WM6BKU2A6RAYQCIE,2020-05-09T03:13:10.000Z,2020-05-09T05:13:10.000Z,6946.0,Assigned,https://www.mturkcontent.com/dynamic/hit?assig...,InternalURL,https://www.mturkcontent.com/dynamic/hit?assig...,0.0,354DQCRRIJZHIYT5G3CFVURWGQJLSW,Install a chrome extension for 7 days ($1 bonu...,A28S2SRZA50N0,HCI Lab,Install a chrome extension that will help you ...,7200.0,2020-05-09T03:01:26.000Z,1.0,2020-05-16T03:01:26.000Z,False,False,2020-05-09T03:01:26.000Z,USD,2.00,/requesters/A28S2SRZA50N0/projects?ref=w_pl_prvw,/projects/354DQCRRIJZHIYT5G3CFVURWGQJLSW/tasks...,/projects/354DQCRRIJZHIYT5G3CFVURWGQJLSW/tasks...
53,54,https://worker.mturk.com/tasks/,INTERNALURL,MTURK,0.0,FINISHED_TASK,2020-05-08T23:22:24.559000,LOGS,ae862298385abab2a0a1619f8cedef9d,3W9XHF7WGLV68SQR2YVUGGPI6QVTK3,3E47SOBEYRW130WM6BKU2A6RAYQCIE,2020-05-09T03:13:10.000Z,2020-05-09T05:13:10.000Z,6676.0,Assigned,https://www.mturkcontent.com/dynamic/hit?assig...,InternalURL,https://www.mturkcontent.com/dynamic/hit?assig...,0.0,354DQCRRIJZHIYT5G3CFVURWGQJLSW,Install a chrome extension for 7 days ($1 bonu...,A28S2SRZA50N0,HCI Lab,Install a chrome extension that will help you ...,7200.0,2020-05-09T03:01:26.000Z,1.0,2020-05-16T03:01:26.000Z,False,False,2020-05-09T03:01:26.000Z,USD,2.00,/requesters/A28S2SRZA50N0/projects?ref=w_pl_prvw,/projects/354DQCRRIJZHIYT5G3CFVURWGQJLSW/tasks...,/projects/354DQCRRIJZHIYT5G3CFVURWGQJLSW/tasks...
89,143,https://worker.mturk.com/tasks/,EXTERNALURL,MTURK,1.0,ADDED_TASK,2020-05-08T23:30:24.605000,LOGS,ae862298385abab2a0a1619f8cedef9d,3KA7IJSNW54MTVXHF3TMHNX8OOTPBI,37FMASSAYCQXZT77SA84XASHHRKIBC,2020-05-09T03:30:06.000Z,2020-05-09T03:42:06.000Z,702.0,Assigned,https://www.pickfu.com/mtjob/T1NYF13WLFI2?assi...,ExternalURL,https://www.pickfu.com/mtjob/T1NYF13WLFI2?assi...,0.0,3D72DZZDWDJ2Q3C2S042BGF3QONYON,"Take a 1-question survey (US-based, Mobile Gam...",AI2HRFAYYSAW7,PickFu,Pick the choice that best answers the question...,720.0,2020-05-09T02:31:43.000Z,1.0,2020-05-12T02:31:43.000Z,False,False,2020-05-09T02:31:43.000Z,USD,0.15,/requesters/AI2HRFAYYSAW7/projects?ref=w_pl_prvw,/projects/3D72DZZDWDJ2Q3C2S042BGF3QONYON/tasks...,/projects/3D72DZZDWDJ2Q3C2S042BGF3QONYON/tasks...
90,151,https://worker.mturk.com/tasks/,EXTERNALURL,MTURK,1.0,FINISHED_TASK,2020-05-08T23:30:54.968000,LOGS,ae862298385abab2a0a1619f8cedef9d,3KA7IJSNW54MTVXHF3TMHNX8OOTPBI,37FMASSAYCQXZT77SA84XASHHRKIBC,2020-05-09T03:30:06.000Z,2020-05-09T03:42:06.000Z,702.0,Assigned,https://www.pickfu.com/mtjob/T1NYF13WLFI2?assi...,ExternalURL,https://www.pickfu.com/mtjob/T1NYF13WLFI2?assi...,0.0,3D72DZZDWDJ2Q3C2S042BGF3QONYON,"Take a 1-question survey (US-based, Mobile Gam...",AI2HRFAYYSAW7,PickFu,Pick the choice that best answers the question...,720.0,2020-05-09T02:31:43.000Z,1.0,2020-05-12T02:31:43.000Z,False,False,2020-05-09T02:31:43.000Z,USD,0.15,/requesters/AI2HRFAYYSAW7/projects?ref=w_pl_prvw,/projects/3D72DZZDWDJ2Q3C2S042BGF3QONYON/tasks...,/projects/3D72DZZDWDJ2Q3C2S042BGF3QONYON/tasks...
558,1185,https://worker.mturk.com/tasks/,EXTERNALURL,MTURK,1.0,ADDED_TASK,2020-05-09T00:05

In [270]:
# Look at the features
df_amt.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17091 entries, 0 to 17090
Data columns (total 36 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   c1                                            17091 non-null  int64  
 1   c2                                            17091 non-null  object 
 2   c3                                            17091 non-null  object 
 3   c5                                            17091 non-null  object 
 4   c6                                            17091 non-null  float64
 5   c7                                            17091 non-null  object 
 6   time                                          17091 non-null  object 
 7   c9                                            17091 non-null  object 
 8   user                                          17091 non-null  object 
 9   c4.task_id                                    160 non-null    obje

In [271]:
# Output continuous/categorical columns to see what the data has.

# df_amt.c3.unique()
# array(['PAGE_LOAD', 'PAGE_BLUR', 'TAB_CHANGE', 'PAGE_FOCUS', 'PAGE_CLICK',
#        'PAGE_SCROLL', 'PAGE_LAST', 'PAGE_CLOSE', 'INTERNALURL',
#        'PAGE_KEY', 'PAGE_INACTIVITY', 'TAB_CLOSED', 'EXTERNALURL',
#        'PAGE_REACTIVATE', 'SYSTEM_DISABLED_WORKING',
#        'SYSTEM_ENABLED_WORKING', 'SYSTEM_ENABLED', 'SYSTEM_DISABLED'],
#       dtype=object)

df_amt.c4.unique()
FINISHED_TASK - ID 54
{
  "accepted_at": "2020-05-09T03:13:10.000Z",
  "assignment_id": "3E47SOBEYRW130WM6BKU2A6RAYQCIE",
  "deadline": "2020-05-09T05:13:10.000Z",
  "expired_task_action_url": "/projects/354DQCRRIJZHIYT5G3CFVURWGQJLSW/tasks?ref=w_pl_prvw",
  "project": {
    "assignable_hits_count": 1,
    "assignment_duration_in_seconds": 7200,
    "caller_meets_preview_requirements": false,
    "caller_meets_requirements": false,
    "creation_time": "2020-05-09T03:01:26.000Z",
    "description": "Install a chrome extension that will help you to keep track of how you spend time on MTurk (potentially helping you to avoid unpaid labor on MTurk).. You are asked to install the chrome extension to track how you are spending your time on the platform. ",
    "hit_requirements": [
      {
        "caller_meets_requirement": null,
        "caller_qualification_value": {
          "integer_value": null,
          "locale_value": {
            "country": null,
            "subdivision": null
          }
        },
        "comparator": "Exists",
        "qualification_type": {
          "description": "Have already used crowd coach plugin",
          "has_test": false,
          "is_requestable": true,
          "keywords": null,
          "name": "CrowdCoach",
          "qualification_type_id": "3WHKV9Z6RB7LBJ77DO4ZLXEIHB2AWU",
          "visibility": true
        },
        "qualification_type_id": "3WHKV9Z6RB7LBJ77DO4ZLXEIHB2AWU",
        "qualification_values": [],
        "worker_action": "ViewHitSet"
      },
      {
        "caller_meets_requirement": null,
        "caller_qualification_value": {
          "integer_value": null,
          "locale_value": {
            "country": null,
            "subdivision": null
          }
        },
        "comparator": "DoesNotExist",
        "qualification_type": {
          "description": "Installation and diagnostic survey",
          "has_test": false,
          "is_requestable": true,
          "keywords": null,
          "name": "GigOverhead Setup Diagnostic",
          "qualification_type_id": "34O6CUUXI0IPA69PLM76WR259RNW0R",
          "visibility": true
        },
        "qualification_type_id": "34O6CUUXI0IPA69PLM76WR259RNW0R",
        "qualification_values": [],
        "worker_action": "ViewHitSet"
      }
    ],
    "hit_set_id": "354DQCRRIJZHIYT5G3CFVURWGQJLSW",
    "last_updated_time": "2020-05-09T03:01:26.000Z",
    "latest_expiration_time": "2020-05-16T03:01:26.000Z",
    "monetary_reward": {
      "amount_in_dollars": 2,
      "currency_code": "USD"
    },
    "requester_id": "A28S2SRZA50N0",
    "requester_name": "HCI Lab",
    "requester_url": "/requesters/A28S2SRZA50N0/projects?ref=w_pl_prvw",
    "title": "Install a chrome extension for 7 days ($1 bonus per each day) to measure your work performance."
  },
  "question": {
    "attributes": {
      "FrameHeight": "0",
      "FrameSourceAttribute": "https://www.mturkcontent.com/dynamic/hit?assignmentId=3E47SOBEYRW130WM6BKU2A6RAYQCIE&amp;hitId=3W9XHF7WGLV68SQR2YVUGGPI6QVTK3&amp;workerId=A3QVZ4SZB79D8W&amp;turkSubmitTo=https%3A%2F%2Fwww.mturk.com"
    },
    "type": "InternalURL",
    "value": "https://www.mturkcontent.com/dynamic/hit?assignmentId=3E47SOBEYRW130WM6BKU2A6RAYQCIE&amp;hitId=3W9XHF7WGLV68SQR2YVUGGPI6QVTK3&amp;workerId=A3QVZ4SZB79D8W&amp;turkSubmitTo=https%3A%2F%2Fwww.mturk.com"
  },
  "state": "Assigned",
  "task_id": "3W9XHF7WGLV68SQR2YVUGGPI6QVTK3",
  "task_url": "/projects/354DQCRRIJZHIYT5G3CFVURWGQJLSW/tasks/3W9XHF7WGLV68SQR2YVUGGPI6QVTK3?assignment_id=3E47SOBEYRW130WM6BKU2A6RAYQCIE&ref=w_pl_prvw",
  "time_to_deadline_in_seconds": 6676
}

# df_amt.c5.unique()
# array(['OTHER', 'MTURK', 'FIVERR', 'UPWORK', 'FREELANCER'], dtype=object)

# df_amt.c6.unique()
# array([ 0.,  1., nan])

# df_amt.c7.unique()
# array(['OTHER', 'TASK_STARTED', 'ADDED_TASK', 'TASK_SUBMITED',
#        'FINISHED_TASK', 'TASKS_LIST', 'WORKER_DASHBOARD', 'UNKNOWN',
#        'TASK_FRAME', 'TASK_PREVIEW', 'TASK_INFO', 'TASK_RETURNED',
#        'PLATFORM_LOGIN', 'TASK_QUEUE', 'TASK_SKIP',
#        'WORKER_EARNINGS_DETAILS', 'TASK_TIMEOUT', 'WORKER_EARNINGS',
#        'WORKER_QUALIFICATIONS', 'TASKS_PER_REQUESTER', 'MESSAGES_SEND',
#        'TASKS_LIST_FILTER', 'WORKER_QUALIFICATIONS_PENDING',
#        'TASKS_PREVIEW', 'PLATFORM_HELP', 'TASKS_PROJECTS',
#        'TASKS_DETAILS', 'MESSAGES_READ', 'TASKS_APPLY'], dtype=object)

# df_amt.c9.unique()
# array(['OTHER', 'WORKING', 'LOGS', 'SEARCHING', 'PROFILE', 'UNKNOWN',
#        'REJECTED', 'COMMUNICATION', 'LEARNING', 'PROPOSAL'], dtype=object)

SyntaxError: invalid syntax (340128650.py, line 12)

## What is the unit of analysis?

## How many observations in total are in the data set? 

## How many unique observations are in the data set? 

## What time period is covered?

## Brief summary of any data cleaning steps you have performed. For example, are there any particular observations / time periods / groups / etc. you have excluded?

## Description of outcome with an appropriate visualization technique.

## Description of key predictors with appropriate visualization techniques that compare predictors to the response. You should investigate all predictors in your data as part of your project. For the purpose of this assignment, pick the one or two predictors that you think are going to be most important in explaining the outcome. Your selection of predictors can either be guided by your domain knowledge or be the result of your EDA on all predictors.

### Cleaning: Reading in the telemetry data (Toloka)
We read in and clean the data from `toloka_telemetry_db.csv`.

**List of available variables (includes target variable `TBD`):**

- **c1**: continuous
- **c2**: url to work task
- **current**: categorial, 31 values ['PAGE_LOAD', 'TAB_CLOSED', 'PAGE_BLUR', 'PAGE_FOCUS', 'TAB_CHANGE', 'CONFIG_UPDATE', 'PLUGIN_INSTALL', 'CONFIG_FILE', 'APP_ACTIVATED', 'PAGE_CLOSE', 'USER', 'BELL_CLICK', 'PAGE_LAST', 'PAGE_CLICK', 'PAGE_KEY', 'PAGE_SCROLL', 'TASK', 'PAGE_INACTIVITY', 'TRAINING', 'PAGE_REACTIVATE', 'LIST_NEW', 'LIST_RECOM', 'LIST_PAY', 'SETT_CLICK', 'MSG_RCV_WORKER', 'MSG_CLICK_WORKER', 'TASK_HIDE_OFF', 'TASK_HIDE_ON', 'MSG_RCV_REQUESTER', 'SETT_SAVE', 'MSG_CLICK_REQUESTER'] - May need to remove/transform urls (e.g. 'https://toloka.yandex.com)
- **event**: json object {activeAssignments, ...} - may include NaN values
- **platform**: categorial, 2 values {0, NaN}
- **subtype**: categorial, 24 values ['TASK_STARTED', 'TASKS_LIST', 'TASK_SUBMITED', 'OTHER', 'FINISHED_TASK', 'SYSTEM', 'GENERAL', 'ADDED_TASK', 'META_DATA', 'UNKNOWN', 'TASK_QUEUE', 'WORKER_QUALIFICATIONS', 'WORKER_DASHBOARD', 'WORKER_EARNINGS', 'WORKER_EARNINGS_DETAILS', 'TASK_INFO', 'MESSAGES_READ', 'TASK_TIMEOUT', 'REFERRAL', 'NOTIFICATIONS', 'MESSAGES_REQUESTER', 'MESSAGES_OUTBOX', 'MESSAGES_ADMIN', 'MESSAGES_NOTIFICATION'] 
- **time**: continous (duration)
- **type**: categorial, 11 values ['WORKING', 'SEARCHING', 'OTHER', 'LOGS', 'CONFIG', 'API', 'SYSTEM', 'UNKNOWN', 'PROFILE', 'COMMUNICATION', 'REJECTED']
- **user**: categorial, user id
- **ordinal**: continuous, 2 values [1, 2]
- **unnamed**: continous, values [nan, 0.00000e+00, 2.89000e+02, ..., 3.79992e+05, 3.25610e+04, 3.61200e+04] – may include NaN values

In [ ]:
# Define columns for data
columns = ['c1', 'c2', 'current', 'event', 'extra', 'platform', 'subtype', 'time', 'type', 'user', 'ordinal', 'unnamed']

# Read the data into a dataframe (this has a header so remove row 0)
df_toloka = pd.read_csv("../data/toloka_telemetry_db.csv", encoding='utf-8', header=0, names=columns)

# Convert the epoch timestamp to datetime
df_toloka['time']=df_toloka.time.map(convert_epoch_time_to_datetime)

# Sort by user then by time
df_toloka = df_toloka.sort_values(by=['user', 'time'], ascending=[True, True])

#Examine the first few rows of the dataframe
df_toloka.head(10)

In [ ]:
# Look at the features
df_toloka.info()

In [ ]:
# Output continuous/categorical columns to see what the data has.

# df_toloka.current.unique()
# array(['PAGE_LOAD', 'TAB_CLOSED', 'PAGE_BLUR', 'PAGE_FOCUS', 'TAB_CHANGE',
#        'CONFIG_UPDATE', 'PLUGIN_INSTALL', 'CONFIG_FILE', 'APP_ACTIVATED',
#        'PAGE_CLOSE', 'USER', 'BELL_CLICK', 'PAGE_LAST', 'PAGE_CLICK',
#        'PAGE_KEY', 'PAGE_SCROLL', 'TASK', 'PAGE_INACTIVITY', 'TRAINING',
#        'PAGE_REACTIVATE', 'LIST_NEW', 'LIST_RECOM', 'LIST_PAY',
#        'SETT_CLICK', 'MSG_RCV_WORKER', 'MSG_CLICK_WORKER',
#        'https://toloka.yandex.com/tasks', 'TASK_HIDE_OFF', 'TASK_HIDE_ON',
#        'MSG_RCV_REQUESTER', 'SETT_SAVE',
#        'https://toloka.yandex.com/vi/tasks',
#        'https://toloka.yandex.com/fr/messages',
#        'https://toloka.yandex.com/es/tasks', 'MSG_CLICK_REQUESTER',
#        'https://toloka.yandex.com/profile/history/73643/2022-01-20',
#        'https://toloka.yandex.com/fr/tasks',
#        'https://toloka.yandex.com/task/31740731?refUuid=75fd7974-884b-4fff-aa42-b08d20fba5b4',
#        'https://toloka.yandex.com/task/21133006/',
#        'https://toloka.yandex.com/tasks/active',
#        'https://toloka.yandex.com/profile/history?status=all',
#        'https://toloka.yandex.com/profile/money',
#        'https://toloka.yandex.com/messages/inbox/620abf1a68ab666108b4b4a0',
#        'https://toloka.yandex.com/task/31161320?refUuid=6acf2a47-f652-4d2a-b1de-4f4c4724c5f6',
#        'https://toloka.yandex.com/task/2990538/00002da1ca--620b8c1db0c49a3a992921d4',
#        'https://toloka.yandex.com/task/31746103/0001e46837--620bb1bec930da47000174d0',
#        'https://toloka.yandex.com/profile/history?status=income',
#        'https://toloka.yandex.com/profile/history?status=blocked',
#        'https://toloka.yandex.com/profile/edit',
# ...
#        'https://toloka.yandex.com/ru/tasks',
#        'https://toloka.yandex.com/ru/task/32969499/',
#        'https://toloka.yandex.com/ru/task/32969361/',
#        'https://toloka.yandex.com/ru/task/33037260',
#        'https://toloka.yandex.com/ru/messages'], dtype=object)


# df_toloka.event.unique()
# array([nan,
#        
#        '{"browserName":"CHROME","currentMode":"PASSIVE","currentState":0,"dailySurveyUrl":"https://docs.google.com/forms/d/e/1FAIpQLSe2XInPGnb9EI539KyUiBNr6gcmRNPzg55LRUaEmXFtx_3zqg/viewform?usp=pp_url&entry.1915115278=","finalSurveyUrl":"https://docs.google.com/forms/d/e/1FAIpQLSefhPyHJs9x-_v0WxQ5aI3kGF3hfXNZH2vk3KMx1C8CnoZSGw/viewform?usp=pp_url&entry.1172212682=","groupId":"GN","hideUnpaidTasks":true,"initialSurveyUrl":"https://docs.google.com/forms/d/e/1FAIpQLSdWQa1AkVc5vV0VhdOYQLGgK77Pw-LEpmsuCBKpdo7QQopWbg/viewform?usp=pp_url&entry.784114392=","installTime":1642708724229,"instructionsUrl":"https://bit.ly/cul-act-gn","isUserStudy":true,"logServerUrl":"https://script.google.com/macros/s/AKfycbwGx2_5a6IwcNI2YZuz2AZvb1J-7Y8Ulk5fYHjZoA8wvHzajv9P55DYiI8UnoV0W403HA/exec","mode":"PROTOCOL","nextDue":1643313524229,"pluginName":"Toloka Assistant","protocol":[{"durationMins":10080,"mode":"PASSIVE"},{"durationMins":10080,"mode":"ACTIVE"},{"durationMins":10080,"mode":"FINISH"}],"rankMethod":"AI","sandbox":false,"serverUrls":["https://script.google.com/macros/s/AKfycbwGx2_5a6IwcNI2YZuz2AZvb1J-7Y8Ulk5fYHjZoA8wvHzajv9P55DYiI8UnoV0W403HA/exec","https://hcilab.ml/overhead/api"],"settings":{"msg_requ":true,"msg_work":true,"not_brow":true,"not_page":true,"not_whil":true,"num_task":5},"socketUrl":"http://hcilab.ml:5000","studyDurationDays":14,"studyDurationMins":20160,"userData":{"acceptedEula":12,"actualUser":{"defaultEmail":"carlostoxtli@yandex.com","displayName":"carlostoxtli","login":"carlostoxtli","readOnlyModeToActUnderAccount":false,"role":"WORKER","uid":1274303000,"userLang":"ES"},"adultAllowed":true,"authoritiesInfo":{"issuedAuthorities":["APP_USER","U_WALLETS_EDIT","U_ASSIGNMENTS_VIEW","U_TRANSACTIONS_CREATE","U_ASSIGNMENTS_UNDERTAKE","APP","U_ASSIGNMENTS_HISTORY","U_PROFILE_VIEW","U_TRANSACTIONS_VIEW","U_ASSIGNMENTS_SUBMIT","U_FORUM_VIEW","U_MESSAGES_CREATE","U_FORUM_EDIT","U_PROFILE_EDIT","U_MESSAGES_VIEW"],"notIssuedAuthoritiesReasons":{}},"availableAccounts":[],"balance":0,"birthDay":"1982-11-15","blockedBalance":0,"citizenship":"US","cityId":103027,"country":"US","createdDate":"2020-12-21","defaultEmail":"carlostoxtli@yandex.com","displayName":"carlostoxtli","education":"HIGH","firstName":"Carlos","fullName":"Carlos Toxtli","gender":"MALE","isAccountOwner":true,"languages":["EN","ES"],"lastName":"Toxtli","login":"carlostoxtli","rating":0,"regionId":223,"role":"WORKER","systemBan":false,"uid":1274303000,"userLang":"ES"},"userId":"a5d84fcd0637d31f4675cdf17b71a35"}',
#        ...,
#        '{"refUuid":"4eb5cecf-42ce-4580-b89d-c2edb9624873","groupUuid":"2cacd9db-5da7-4ff4-9032-d38402d5825d","lightweightTec":{"poolId":33139449,"projectId":58019,"poolStartedAt":"2022-04-25T10:00:27.146","mayContainAdultContent":true,"title":"Find content from website (universal app prod)","description":"Find the exact web page that contains the listed information from the given website domain. Use the google translate or bing translate browser extensions to translate international web pages into understandable language.\\\\nНайдите точную веб-страницу, содержащую перечисленную информацию из данного домена веб-сайта. Используйте расширения браузера google translate или bing translate, чтобы переводить международные веб-страницы на понятный язык.\\\\n","hasInstructions":true,"snapshotMajorVersion":1,"snapshotMinorVersion":8,"snapshotMajorVersionActual":true,"assignmentConfig":{"reward":"0.020","maxDurationSeconds":600,"issuing":{"type":"AUTOMATIC"}},"trainingConfig":{"training":false},"requesterInfo":{"id":"97e0e18092318a1140eb08402e7cc5ac","name":{"EN":"Bing Local Search 2","FR":"Bing Local Search 2","ID":"Bing Local Search 2","RU":"Bing Local Search 2","TR":"Bing Local Search 2"},"trusted":false},"projectMetaInfo":{"projectId":58019,"bookmarked":true,"bookmarkedAt":"2022-03-12T01:28:35.577","experimentMeta":{"dj_task_duration__snippet__duration_less_than_minute":"1","dj_project_class__snippet__web_searching":"1","dj_project_tag__requester_type__snippet__experienced_requester":"1"}},"iframeSubdomain":"97e0e18092318a1140eb08402e7cc5ac"},"availability":{"available":true},"activeAssignments":[{"id":"0001f9aaf9--6266ca43ca6f212f45a79130","expireTime":"2022-04-25T16:30:19.528","secondsLeft":597,"reward":0.02}],"acceptanceDetails":{"postAccept":true,"acceptanceRate":99,"acceptancePeriodDays":1,"averageAcceptancePeriodDays":1},"trainingDetails":{"training":false},"taskDetails":{"grade":{"total_grade":4.87},"averageSubmitTimeSec":23,"averageAcceptanceTimeSec":86406,"moneyAvgHourly":3.13043472,"moneyAvg":17.48184971098265,"moneyMed":18.22,"moneyTop10":30.288000000000004,"moneyMax3":18.330940090548125},"grade":{"total_grade":4.87}}',
#        '{"refUuid":"e046bcc5-ece8-40b1-8203-e48fed3ddc99","groupUuid":"9fde347c-5a79-4e78-9023-fe325f8bd616","lightweightTec":{"poolId":33139449,"projectId":58019,"poolStartedAt":"2022-04-25T10:00:27.146","mayContainAdultContent":true,"title":"Find content from website (universal app prod)","description":"Find the exact web page that contains the listed information from the given website domain. Use the google translate or bing translate browser extensions to translate international web pages into understandable language.\\\\nНайдите точную веб-страницу, содержащую перечисленную информацию из данного домена веб-сайта. Используйте расширения браузера google translate или bing translate, чтобы переводить международные веб-страницы на понятный язык.\\\\n","hasInstructions":true,"snapshotMajorVersion":1,"snapshotMinorVersion":8,"snapshotMajorVersionActual":true,"assignmentConfig":{"reward":"0.020","maxDurationSeconds":600,"issuing":{"type":"AUTOMATIC"}},"trainingConfig":{"training":false},"requesterInfo":{"id":"97e0e18092318a1140eb08402e7cc5ac","name":{"EN":"Bing Local Search 2","FR":"Bing Local Search 2","ID":"Bing Local Search 2","RU":"Bing Local Search 2","TR":"Bing Local Search 2"},"trusted":false},"projectMetaInfo":{"projectId":58019,"bookmarked":true,"bookmarkedAt":"2022-03-12T01:28:35.577","experimentMeta":{"dj_task_duration__snippet__duration_less_than_minute":"1","dj_project_class__snippet__web_searching":"1","dj_project_tag__requester_type__snippet__experienced_requester":"1"}},"iframeSubdomain":"97e0e18092318a1140eb08402e7cc5ac"},"availability":{"available":true},"activeAssignments":[{"id":"0001f9aaf9--6266ca43ca6f212f45a79130","expireTime":"2022-04-25T16:30:19.528","secondsLeft":567,"reward":0.02}],"acceptanceDetails":{"postAccept":true,"acceptanceRate":99,"acceptancePeriodDays":1,"averageAcceptancePeriodDays":1},"trainingDetails":{"training":false},"taskDetails":{"grade":{"total_grade":4.87},"averageSubmitTimeSec":23,"averageAcceptanceTimeSec":86406,"moneyAvgHourly":3.13043472,"moneyAvg":17.48184971098265,"moneyMed":18.22,"moneyTop10":30.288000000000004,"moneyMax3":18.330940090548125},"grade":{"total_grade":4.87}}',
#        '{"uid":1206161147,"login":"sholesy@gmail.com","role":"WORKER","userLang":"EN","defaultEmail":"sholesy@gmail.com","connectionId":"s:1650784330835:uZlwaQ:2d","authorizationStatus":"VALID","avatarId":"0/0-0","displayName":"sholesy@gmail.com","fullName":"Oluwatosin Solesi","firstName":"Oluwatosin","lastName":"Solesi","isAccountOwner":true,"actualUser":{"uid":1206161147,"login":"sholesy@gmail.com","role":"WORKER","userLang":"EN","defaultEmail":"sholesy@gmail.com","displayName":"sholesy@gmail.com","readOnlyModeToActUnderAccount":false},"availableAccounts":[],"createdDate":"2020-10-26","systemBan":false,"gender":"FEMALE","birthDay":"1991-10-14","cityId":21063,"country":"NG","citizenship":"US","education":"HIGH","languages":["EN"],"adultAllowed":true,"acceptedEula":13,"rating":0,"authoritiesInfo":{"issuedAuthorities":["U_ASSIGNMENTS_VIEW","U_ASSIGNMENTS_HISTORY","APP_USER","U_MESSAGES_CREATE","U_MESSAGES_VIEW","U_FORUM_VIEW","U_PROFILE_VIEW","U_WALLETS_EDIT","U_TRANSACTIONS_VIEW","U_FORUM_EDIT","U_ASSIGNMENTS_UNDERTAKE","U_TRANSACTIONS_CREATE","U_ASSIGNMENTS_SUBMIT","U_PROFILE_EDIT","APP"],"notIssuedAuthoritiesReasons":{}},"balance":"0.191","blockedBalance":"0.035","regionId":20741}'],
#       dtype=object)
# {
#   "acceptanceDetails": {
#     "postAccept": false
#   },
#   "activeAssignments": [
#     {
#       "expireTime": "2022-01-20T19:37:28.194",
#       "id": "00001086d6--61e9b930a2d62b2b56644596",
#       "reward": 0.3,
#       "secondsLeft": 193
#     }
#   ],
#   "availability": {
#     "available": true
#   },
#   "groupUuid": "9455a911-5624-4951-9f4e-ea09cc1cc5f5",
#   "lightweightTec": {
#     "assignmentConfig": {
#       "issuing": {
#         "type": "AUTOMATIC"
#       },
#       "maxDurationSeconds": 200,
#       "reward": 0.3
#     },
#     "description": "Answer the questions in the survey. Choose one or more options or write your own answer",
#     "hasInstructions": true,
#     "iframeSubdomain": "54f8685950e9694b99faccce011a21df",
#     "mayContainAdultContent": false,
#     "poolId": 1083094,
#     "poolStartedAt": "2022-01-20T19:01:57.121",
#     "projectId": 89244,
#     "projectMetaInfo": {
#       "experimentMeta": {},
#       "projectId": 89244
#     },
#     "requesterInfo": {
#       "id": "54f8685950e9694b99faccce011a21df",
#       "name": {
#         "EN": "davidjohnsonits"
#       },
#       "trusted": false
#     },
#     "snapshotMajorVersion": 1,
#     "snapshotMajorVersionActual": true,
#     "snapshotMinorVersion": 2,
#     "title": "Survey One David",
#     "trainingConfig": {
#       "training": false
#     }
#   },
#   "refUuid": "a95bab84-7c39-45bc-9268-2f474011c0ae",
#   "taskDetails": {
#     "averageSubmitTimeSec": 13,
#     "moneyAvgHourly": 83.07692316
#   },
#   "trainingDetails": {
#     "training": false
#   }
# }

# df_toloka.platform.unique()
# array([ 0., nan])

# df_toloka.subtype.unique()
# array(['TASK_STARTED', 'TASKS_LIST', 'TASK_SUBMITED', 'OTHER',
#        'FINISHED_TASK', 'SYSTEM', 'GENERAL', 'ADDED_TASK', 'META_DATA',
#        'UNKNOWN', 'TASK_QUEUE', 'WORKER_QUALIFICATIONS',
#        'WORKER_DASHBOARD', 'WORKER_EARNINGS', 'WORKER_EARNINGS_DETAILS',
#        'TASK_INFO', 'MESSAGES_READ', 'TASK_TIMEOUT', 'REFERRAL',
#        'NOTIFICATIONS', 'MESSAGES_REQUESTER', 'MESSAGES_OUTBOX',
#        'MESSAGES_ADMIN', 'MESSAGES_NOTIFICATION'], dtype=object)

# df_toloka.type.unique()
# array(['WORKING', 'SEARCHING', 'OTHER', 'LOGS', 'CONFIG', 'API', 'SYSTEM',
#        'UNKNOWN', 'PROFILE', 'COMMUNICATION', 'REJECTED'], dtype=object)

# df_toloka.ordinal.unique()
# array([1, 2])

# df_toloka.unnamed.unique()
# array([        nan, 0.00000e+00, 2.89000e+02, ..., 3.79992e+05,
#        3.25610e+04, 3.61200e+04])

## What is the unit of analysis?

## How many observations in total are in the data set? 

## How many unique observations are in the data set? 

## What time period is covered?

## Brief summary of any data cleaning steps you have performed. For example, are there any particular observations / time periods / groups / etc. you have excluded?

## Description of outcome with an appropriate visualization technique.

## Description of key predictors with appropriate visualization techniques that compare predictors to the response. You should investigate all predictors in your data as part of your project. For the purpose of this assignment, pick the one or two predictors that you think are going to be most important in explaining the outcome. Your selection of predictors can either be guided by your domain knowledge or be the result of your EDA on all predictors.